Build a model to convert embeddings to a sentiment

In [11]:
#!pip install -U scikit-learn
#!pip install pandas

In [1]:
import io
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

Obtain a training data set

ChatGPT: """
Please generate 100 headlines from a newspaper and assign it a label: optimistic, pessimistic, neutral. Please present the data in csv format.
"""

Result:

```csv
Headline,Label
"Breakthrough in Cancer Treatment Shows Promising Results",Optimistic
"Global Poverty Rates Hit Historic Lows",Optimistic
"Scientists Announce Discovery of a Clean Energy Source",Optimistic
"Local Community Raises $1 Million for Children’s Hospital",Optimistic
"Job Growth Surges Across the Nation",Optimistic
"Endangered Species Makes Historic Comeback",Optimistic
"High School Students Win International Robotics Competition",Optimistic
"Solar Energy Now Cheaper Than Fossil Fuels",Optimistic
"New Vaccine Reduces Risk of Malaria by 90%",Optimistic
"Neighborhood Garden Initiative Feeds Hundreds Weekly",Optimistic
"International Peace Talks Yield Promising Agreement",Optimistic
"Startup Revolutionizes Water Purification in Remote Areas",Optimistic
"School Literacy Rates Improve by 20% in Five Years",Optimistic
"City Park Redevelopment Wins Environmental Award",Optimistic
"Volunteers Plant 1 Million Trees Across the Region",Optimistic
"Electric Car Sales Outpace Gasoline Models for the First Time",Optimistic
"AI Predicts Earthquakes with High Accuracy, Saves Lives",Optimistic
"First Successful Mission to Remove Space Debris Completed",Optimistic
"Renewable Energy Jobs Outnumber Fossil Fuel Jobs for First Time",Optimistic
"College Tuition Costs Decrease Across Multiple States",Optimistic
"Scientists Report Ozone Layer Healing Faster Than Expected",Optimistic
"New Law Protects Ocean Ecosystems from Overfishing",Optimistic
"Teen Creates App to Help the Elderly with Daily Tasks",Optimistic
"Global Literacy Rates Reach All-Time High",Optimistic
"Affordable Housing Initiative Gains Momentum in Urban Areas",Optimistic
"Wildfires Destroy Thousands of Acres, Displacing Communities",Pessimistic
"Unemployment Rates Surge Amid Economic Uncertainty",Pessimistic
"Endangered Species Declared Extinct Due to Habitat Loss",Pessimistic
"Crime Rates in Major Cities Reach Decade High",Pessimistic
"Polar Ice Caps Melting at Alarming Rates, Report Finds",Pessimistic
"Government Announces Cuts to Education Funding",Pessimistic
"Rising Inflation Worsens Cost of Living Crisis",Pessimistic
"Prolonged Drought Threatens Crops in Agricultural Regions",Pessimistic
"Cyberattack Exposes Data of Millions of Users",Pessimistic
"Public Health Officials Warn of New Viral Outbreak",Pessimistic
"Major Oil Spill Devastates Marine Life Along Coast",Pessimistic
"Political Tensions Escalate Between Neighboring Countries",Pessimistic
"Scientists Warn of Impending Global Water Shortage",Pessimistic
"Mass Layoffs Announced by Leading Tech Companies",Pessimistic
"Hurricane Leaves Thousands Without Power and Homes",Pessimistic
"Ocean Temperatures Hit Record Highs, Threatening Ecosystems",Pessimistic
"Famine Spreads in War-Torn Region, Millions at Risk",Pessimistic
"Protests Erupt Nationwide Over Economic Inequality",Pessimistic
"New Study Shows Decline in Global Biodiversity",Pessimistic
"Healthcare System Overwhelmed as Demand Surges",Pessimistic
"City Struggles to Recover After Devastating Earthquake",Pessimistic
"Corruption Scandal Rocks National Government",Pessimistic
"Air Pollution in Urban Areas Hits Dangerous Levels",Pessimistic
"Flooding Displaces Thousands After Heavy Rainfall",Pessimistic
"Space Agency Delays Mission Due to Technical Issues",Neutral
"New CEO Appointed at Fortune 500 Company",Neutral
"City Council Debates Controversial Zoning Proposal",Neutral
"Quarterly Economic Report Shows Mixed Results",Neutral
"Research Team Publishes Findings on Climate Change Trends",Neutral
"Experts Discuss the Future of Autonomous Vehicles",Neutral
"State Introduces Pilot Program for Electric Buses",Neutral
"Local University Launches New Data Science Program",Neutral
"Annual Report Highlights Industry Growth and Challenges",Neutral
"Retail Sales Slightly Higher Than Last Quarter",Neutral
"Neighborhood Farmers Market Returns After Winter Break",Neutral
"City Implements New Parking Rules Starting Next Month",Neutral
"Study Shows Changes in Consumer Spending Habits",Neutral
"Tech Conference Attracts Innovators From Around the World",Neutral
"New Policy Requires Improved Safety Standards in Factories",Neutral
"City Expands Bike Lanes, Receives Mixed Reactions",Neutral
"Energy Company Announces Plans for Offshore Wind Farms",Neutral
"Traffic Congestion Expected Due to Road Repairs",Neutral
"Weather Forecast Predicts Mild Summer Ahead",Neutral
"Tourist Numbers Steady in Historic Downtown Area",Neutral
"Country’s Currency Value Stabilizes After Recent Drop",Neutral
"Local Business Owner Opens Second Restaurant",Neutral
"Public Transit Authority Introduces New Fare Options",Neutral
"Government to Announce Budget Next Week",Neutral
"Experts Call for More Research Into Renewable Energy",Neutral
"Construction Delayed on New High-Speed Rail Project",Neutral
"New Art Exhibit Opens at City Museum",Neutral
"Education Department Rolls Out Standardized Test Updates",Neutral
"Festival Draws Thousands Despite Rainy Weather",Neutral
"Tech Startups See Growth Amid Global Challenges",Neutral

```

In [2]:
generated_training_data = """
Headline,Label
"Breakthrough in Cancer Treatment Shows Promising Results",Optimistic
"Global Poverty Rates Hit Historic Lows",Optimistic
"Scientists Announce Discovery of a Clean Energy Source",Optimistic
"Local Community Raises $1 Million for Children’s Hospital",Optimistic
"Job Growth Surges Across the Nation",Optimistic
"Endangered Species Makes Historic Comeback",Optimistic
"High School Students Win International Robotics Competition",Optimistic
"Solar Energy Now Cheaper Than Fossil Fuels",Optimistic
"New Vaccine Reduces Risk of Malaria by 90%",Optimistic
"Neighborhood Garden Initiative Feeds Hundreds Weekly",Optimistic
"International Peace Talks Yield Promising Agreement",Optimistic
"Startup Revolutionizes Water Purification in Remote Areas",Optimistic
"School Literacy Rates Improve by 20% in Five Years",Optimistic
"City Park Redevelopment Wins Environmental Award",Optimistic
"Volunteers Plant 1 Million Trees Across the Region",Optimistic
"Electric Car Sales Outpace Gasoline Models for the First Time",Optimistic
"AI Predicts Earthquakes with High Accuracy, Saves Lives",Optimistic
"First Successful Mission to Remove Space Debris Completed",Optimistic
"Renewable Energy Jobs Outnumber Fossil Fuel Jobs for First Time",Optimistic
"College Tuition Costs Decrease Across Multiple States",Optimistic
"Scientists Report Ozone Layer Healing Faster Than Expected",Optimistic
"New Law Protects Ocean Ecosystems from Overfishing",Optimistic
"Teen Creates App to Help the Elderly with Daily Tasks",Optimistic
"Global Literacy Rates Reach All-Time High",Optimistic
"Affordable Housing Initiative Gains Momentum in Urban Areas",Optimistic
"Wildfires Destroy Thousands of Acres, Displacing Communities",Pessimistic
"Unemployment Rates Surge Amid Economic Uncertainty",Pessimistic
"Endangered Species Declared Extinct Due to Habitat Loss",Pessimistic
"Crime Rates in Major Cities Reach Decade High",Pessimistic
"Polar Ice Caps Melting at Alarming Rates, Report Finds",Pessimistic
"Government Announces Cuts to Education Funding",Pessimistic
"Rising Inflation Worsens Cost of Living Crisis",Pessimistic
"Prolonged Drought Threatens Crops in Agricultural Regions",Pessimistic
"Cyberattack Exposes Data of Millions of Users",Pessimistic
"Public Health Officials Warn of New Viral Outbreak",Pessimistic
"Major Oil Spill Devastates Marine Life Along Coast",Pessimistic
"Political Tensions Escalate Between Neighboring Countries",Pessimistic
"Scientists Warn of Impending Global Water Shortage",Pessimistic
"Mass Layoffs Announced by Leading Tech Companies",Pessimistic
"Hurricane Leaves Thousands Without Power and Homes",Pessimistic
"Ocean Temperatures Hit Record Highs, Threatening Ecosystems",Pessimistic
"Famine Spreads in War-Torn Region, Millions at Risk",Pessimistic
"Protests Erupt Nationwide Over Economic Inequality",Pessimistic
"New Study Shows Decline in Global Biodiversity",Pessimistic
"Healthcare System Overwhelmed as Demand Surges",Pessimistic
"City Struggles to Recover After Devastating Earthquake",Pessimistic
"Corruption Scandal Rocks National Government",Pessimistic
"Air Pollution in Urban Areas Hits Dangerous Levels",Pessimistic
"Flooding Displaces Thousands After Heavy Rainfall",Pessimistic
"Space Agency Delays Mission Due to Technical Issues",Neutral
"New CEO Appointed at Fortune 500 Company",Neutral
"City Council Debates Controversial Zoning Proposal",Neutral
"Quarterly Economic Report Shows Mixed Results",Neutral
"Research Team Publishes Findings on Climate Change Trends",Neutral
"Experts Discuss the Future of Autonomous Vehicles",Neutral
"State Introduces Pilot Program for Electric Buses",Neutral
"Local University Launches New Data Science Program",Neutral
"Annual Report Highlights Industry Growth and Challenges",Neutral
"Retail Sales Slightly Higher Than Last Quarter",Neutral
"Neighborhood Farmers Market Returns After Winter Break",Neutral
"City Implements New Parking Rules Starting Next Month",Neutral
"Study Shows Changes in Consumer Spending Habits",Neutral
"Tech Conference Attracts Innovators From Around the World",Neutral
"New Policy Requires Improved Safety Standards in Factories",Neutral
"City Expands Bike Lanes, Receives Mixed Reactions",Neutral
"Energy Company Announces Plans for Offshore Wind Farms",Neutral
"Traffic Congestion Expected Due to Road Repairs",Neutral
"Weather Forecast Predicts Mild Summer Ahead",Neutral
"Tourist Numbers Steady in Historic Downtown Area",Neutral
"Country’s Currency Value Stabilizes After Recent Drop",Neutral
"Local Business Owner Opens Second Restaurant",Neutral
"Public Transit Authority Introduces New Fare Options",Neutral
"Government to Announce Budget Next Week",Neutral
"Experts Call for More Research Into Renewable Energy",Neutral
"Construction Delayed on New High-Speed Rail Project",Neutral
"New Art Exhibit Opens at City Museum",Neutral
"Education Department Rolls Out Standardized Test Updates",Neutral
"Festival Draws Thousands Despite Rainy Weather",Neutral
"Tech Startups See Growth Amid Global Challenges",Neutral
"""

In [3]:
training_data_df = pd.read_csv(io.StringIO(generated_training_data))
training_data_df.head()

,Headline,Label
0,Breakthrough in Cancer Treatment Shows Promisi...,Optimistic
1,Global Poverty Rates Hit Historic Lows,Optimistic
2,Scientists Announce Discovery of a Clean Energ...,Optimistic
3,Local Community Raises $1 Million for Children...,Optimistic
4,Job Growth Surges Across the Nation,Optimistic


In [4]:
training_data_df = training_data_df.sample(frac=1)
training_data_df.head()

,Headline,Label
56,Local University Launches New Data Science Pro...,Neutral
1,Global Poverty Rates Hit Historic Lows,Optimistic
58,Retail Sales Slightly Higher Than Last Quarter,Neutral
25,"Wildfires Destroy Thousands of Acres, Displaci...",Pessimistic
39,Hurricane Leaves Thousands Without Power and H...,Pessimistic


score

In [5]:
model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\yanch\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yanch\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
%%time
embeddings = model.encode(training_data_df.Headline)

CPU times: total: 188 ms
Wall time: 521 ms


In [30]:
#embeddings = np.load('headlines_chicagotribune_2024-12-01.npy')

In [31]:
#embeddings.shape

In [32]:
#embeddings[0].shape

In [33]:
# we don't need these embeddings, we just need the test dataset we creat to train the model :)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(embeddings, training_data_df.Label, test_size=0.2) 

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((63, 384), (16, 384), (63,), (16,))

In [10]:
from sklearn import svm
from sklearn import metrics

In [11]:
clf = svm.SVC(kernel='linear') # Linear Kernel

In [12]:
%%time
clf.fit(X_train, y_train)

CPU times: total: 0 ns
Wall time: 16.6 ms


SVC(kernel='linear')

In [13]:
y_pred = clf.predict(X_test)

In [14]:
metrics.accuracy_score(y_test, y_pred)

0.375

In [15]:
metrics.precision_score(y_test, y_pred, average='weighted')

0.25

In [16]:
metrics.recall_score(y_test, y_pred, average='weighted')

0.375

In [17]:
import joblib

In [18]:
joblib.dump(clf, 'svm.joblib')

['svm.joblib']

In [19]:
from joblib import load
from sentence_transformers import SentenceTransformer

# Load pre-trained model and vectorizer
svm_model = load('svm.joblib')  # path to your SVM model file
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # Or use the same model you trained with

# Example headlines to test
test_headlines = [
    "Breakthrough in solar panel efficiency announced",
    "Major layoffs reported in tech industry",
    "City implements new recycling program to reduce waste"
]

# Convert headlines to embeddings
X_test = embedding_model.encode(test_headlines)

# Predict sentiment
predictions = svm_model.predict(X_test)

# Print results
for headline, label in zip(test_headlines, predictions):
    print(f"{headline} --> {label}")


Breakthrough in solar panel efficiency announced --> Neutral
Major layoffs reported in tech industry --> Neutral
City implements new recycling program to reduce waste --> Neutral
